In [ ]:
# install requirements
import sys

#ran once for installation
# !pip3 install transformers==4.15.0 timm==0.4.12 fairscale==0.4.4
# !git clone https://github.com/salesforce/BLIP
# !pip install torchvision

%cd BLIP

/home/robotics1/Desktop/image-passage-answering-MuMuQA/BLIP


In [ ]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_demo_image(image_size,device,imagefile):
    img_url = imagefile
    #the following is used when url is used for an imagefile
    #raw_image = Image.open(requests.get(imagefile, stream=True).raw).convert('RGB')   
    raw_image = Image.open(imagefile).convert('RGB') 

    w,h = raw_image.size
    #display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
from models.blip_itm import blip_itm
image_size = 384
#url = "http://images.cocodataset.org/val2017/000000039769.jpg" #get an image file name and its path

imagepath = '/home/robotics1/Desktop/image-passage-answering-MuMuQA/BLIP/'
image1 = imagepath+"cats.png" #specify the image file name

url = image1

image = load_demo_image(image_size=image_size,device=device, imagefile=url)

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth'
    
model = blip_itm(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device='cpu')

#specify each question and its answer choices
question = "What are the aminals in the image?"
answerchoices = ["pigs", "dogs", "cats", "monkeys"]
# question = "How many animals are in the image?"
# answerchoices = ["one", "two", "three", "four"]

#Define a function to merge the question and each answer
def merge_string(question, answer):
    text1 = question + " " + answer
    return text1

#Create a list of multiple texts, each of them containing the question and each answer
textlist = []
for eachanswer in answerchoices:   
    textlist.append(merge_string(question,eachanswer))

print(textlist)

#Use BLIP to process the given image and multiple texts. The scores will show their relevance between the image and each text.
scorelist=[]
for i in textlist:
  itm_output = model(image,i,match_head='itm')
  scorelist.append(float(torch.nn.functional.softmax(itm_output,dim=1)[:,1]))

print(scorelist)

#Check which answer choice has the highest score. 
#the final answer index is the answer given by the model. 
#Check if this is sam/home/robotics1/Desktop/image-passage-answering-MuMuQA/BLIP/cats.png
scoremax= max(scorelist)
print('final answer: ', answerchoices[scorelist.index(scoremax)])
print('final answer index: ', scorelist.index(scoremax))


load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth
['What are the aminals in the image? pigs', 'What are the aminals in the image? dogs', 'What are the aminals in the image? cats', 'What are the aminals in the image? monkeys']
[0.00709642143920064, 0.010806050151586533, 0.2144784927368164, 0.00195474736392498]
final answer:  cats
final answer index:  2


In [ ]:
#MUMUQA
import json
import csv

f = open('MANUALLY EDITTED randomized_MuMuQA.json')
model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth'
model = blip_itm(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device='cpu')
initial_path = "/home/robotics1/Desktop/train/imageDownloading"
  
data = json.load(f)
print(data[0])


load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth
{'qid': 0, 'images': './images/459585D-D9DF-4CF3-B7C9-38A2C3267B1D.jpg.png', 'multiple_images': False, 'passage': 'Nelson Mandela wrote about feeling, upon his release from prison, that he had failed to be there for his own children and grandchildren. Instead, he wrote, his decades-long struggle against apartheid had turned him into the father of his nation.\nFormer South African President Nelson Mandela as he celebrates his birthday with family in Qunu, South Africa, July 18, 2012.\nThat became clear this weekend as news of his Saturday hospitalization spread and the Internet buzzed with virtual get-well cards from luminaries, journalists and ordinary South Africans.\nMandela spent Saturday night in the hospital and appeared “comfortable and in good care” on Sunday morning, said President Jacob Zuma.\n Former South African President Nelson Mandela as he celebrates

In [ ]:

error_qid_list = []

with open('MUMUQA_image_passage_QAP.csv', mode='w', newline='') as csvfile:
   writer = csv.writer(csvfile)

   for index in range(0, len(data)):
   # for index in range(4401,len(data)):
      print(index)

      qid = data[index]['qid']
      correct_answer = data[index]["answer"]
      file_specific_path = data[index]['images'][1:]
      final_path = initial_path + file_specific_path
      #print(final_path)
      passage = data[index]['passage']


      url = final_path

      try:
         image = load_demo_image(image_size=image_size,device=device, imagefile=url)
      except Exception as e:
         print(f"Error occurred: {e}")
         error_qid_list.append(index)
         continue


         


      question =  data[index]['question'] + "?"
      answer_choices = data[index]['answer_choices']
      # print(question)
      # print(answer_choices)
      text_list = []
      for eachanswer in answer_choices:
         text_list.append(merge_string(question,eachanswer) + ". "+ passage) # answering questions as question, answer, passage

      #print(text_list)

      score_list=[]
      for i in text_list:
         itm_output = model(image,i,match_head='itm')
         score_list.append(float(torch.nn.functional.softmax(itm_output,dim=1)[:,1]))

      #print(score_list)

      score_max= max(score_list)
      #print('final answer for index ', index,': ', answer_choices[score_list.index(score_max)])
      #print('final answer index: ', score_list.index(score_max))

      #list [qid, question, answer from JSON file, model generated answer index, model-generated answer text]
      appendList = [qid, question, correct_answer, score_list.index(score_max), answer_choices[score_list.index(score_max)]]
      #print(appendList)
      writer.writerow(appendList)

error_file = open("error_file_QAP.txt", "w")
for error_qid in error_qid_list:
   error_file.write(str(error_qid) +"\n")
error_file.close()

4401
4402
4403
4404
4405
4406
4407
4408
4409
4410
4411
4412
4413
4414
4415
4416
4417
4418
4419
4420
4421
4422
4423
4424
4425
4426
4427
4428
4429
4430
4431
4432
4433
4434
4435
4436
4437
4438
4439
4440
4441
4442
4443
4444
4445
4446
4447
4448
4449
4450
4451
4452
4453
4454
4455
4456
4457
4458
4459
4460
4461
4462
4463
4464
4465
4466
4467
4468
4469
4470
4471
4472
4473
4474
4475
4476
4477
4478
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4493
4494
4495
4496
4497
4498
4499
4500
4501
4502
4503
4504
4505
4506
4507
4508
4509
4510
4511
4512
4513
4514
4515
4516
4517
4518
4519
4520
4521
4522
4523
4524
4525
4526
4527
4528
4529
4530
4531
4532
4533
4534
4535
4536
4537
4538
4539
4540
4541
4542
4543
4544
4545
4546
4547
4548
4549
4550
4551
4552
4553
4554
4555
4556
4557
4558
4559
4560
4561
4562
4563
4564
4565
4566
4567
4568
4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600


In [ ]:
print(len(data))
print(data[1151]['qid'])
print(data[4401])

5285
1151
{'qid': 4401, 'images': './images/AED6FBA-43E7-4EB3-8CFD-E7D98779B72D.jpg.png', 'multiple_images': False, 'passage': "The existing backlog of immigration applications has had an unusually large impact on Asian-American citizens, for whom family reunions represent a primary path to immigration.\nBill supporters, including the Pacific American Labor Alliance, the National Council of Asian Pacific Americans, Asians Advancing Justice, Immigration Equality, and the Lutheran Immigration and Refugee Service, say that without reform, it will take the current immigration system nearly 20 years to cycle through the current backlog of unresolved visa applications.\nRep. Honda's office says the RFA is supported by more than 65 legislators in the House.\nThis report was produced in collaboration with VOA's Mandarin Service.  U.S. Representative Mike Honda (D-Calif.), talks about the Reuniting Families Act at a March 17, 2016 news conference on Capitol Hill.", 'question': 'how many members

In [ ]:
print(answer_choices)
print(text_list)

['Milal Mission', 'Silal Mission', 'Swalal Mission ', 'Wolal Mission ']
['who did woman in the image go on american idol to help? Milal Mission. He said, “good things come to those who do good, and provide unselfishly for others.” He says his Christian faith keeps him grounded and that he prays before each concert. \nHan said he entered the American Idol contest not to become a star but to promote Milal Mission, a non-profit organization in Korea that helps disabled youth. Han suffered from depression in his early years and went to the organization for help. He said that working with them turned his life around and that he wanted to give back to those who had given him so much. \n“When I was in Korea, I pursued my dream [of making it as a singer] for two years but nothing happened, so I had depression.  Han is the highest ranked Korean-American ever to appear on “American Idol” (Courtesy Fox "American Idol")', 'who did woman in the image go on american idol to help? Silal Mission. He s